In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import timedelta
import xlsxwriter

# ==========================================
# 策略參數與設定
# ==========================================
INITIAL_CAPITAL = 50_000_000
MAX_HOLDINGS = 3
REBALANCE_INTERVAL = 5  # 每5天
FILE_PATH = 'data/data.xlsx'
RESULT_EXCEL = 'strategy_results.xlsx'
REPRODUCE_MD = 'reproduce_strategy.md'

# ==========================================
# 資料載入與預處理
# ==========================================
def load_and_preprocess_data(filepath):
    df = pd.read_excel(filepath, header=[0, 1])
    
    # 基於 Level 0 (Ticker) 去重
    tickers_all = df.columns.get_level_values(0)
    df = df.loc[:, ~tickers_all.duplicated()]
    
    dates = pd.to_datetime(df[('日期', '日期')])
    prices = df.drop(columns=[('日期', '日期')])
    
    tickers = prices.columns.get_level_values(0)
    names = prices.columns.get_level_values(1)
    ticker_to_name = dict(zip(tickers, names))
    
    prices.columns = tickers
    prices.index = dates
    prices = prices.ffill()
    
    return prices, ticker_to_name

# ==========================================
# 完整的交易追蹤版本
# ==========================================
def run_backtest_full(prices, sma_period, ticker_to_name, initial_capital=INITIAL_CAPITAL):
    sma = prices.rolling(window=sma_period).mean()
    roc = (prices / prices.shift(sma_period)) - 1
    dates = prices.index
    equity = pd.Series(index=dates, data=0.0)
    equity.iloc[:sma_period] = initial_capital
    cash = initial_capital
    holdings = {} 
    
    closed_trades = []
    daily_holdings_log = []
    daily_records = []
    
    target_tickers = []
    pending_rebalance = False
    signal_date_t = None

    for i in range(sma_period, len(dates)):
        current_date = dates[i]
        
        current_val = cash
        for t, info in holdings.items():
            current_val += info['shares'] * prices.loc[current_date, t]
        equity.iloc[i] = current_val
        
        if (i - sma_period) % REBALANCE_INTERVAL == 0:
            signal_date_t = current_date
            curr_p = prices.iloc[i]
            curr_sma = sma.iloc[i]
            curr_roc = roc.iloc[i]
            
            mask = (curr_p > curr_sma) & (curr_roc.notna()) & (curr_roc > 0)
            eligible = curr_roc[mask].sort_values(ascending=False)
            target_tickers = eligible.head(MAX_HOLDINGS).index.tolist()
            
            pending_rebalance = True
            daily_records.append({'日期': current_date, '事件': '訊號產生', '細節': f'目標: {target_tickers}'})
        else:
            daily_records.append({'日期': current_date, '事件': '觀察', '細節': ''})

        if pending_rebalance and current_date > signal_date_t:
            trade_date = current_date
            current_holdings_list = list(holdings.keys())
            for t in current_holdings_list:
                info = holdings.pop(t)
                sell_price = prices.loc[trade_date, t]
                closed_trades.append({
                    'Ticker': t, '名稱': ticker_to_name.get(t, t),
                    '買入訊號日': info['buy_signal_date'], '買入日': info['buy_date'],
                    '買入價': info['buy_price'], '賣出訊號日': signal_date_t,
                    '賣出日': trade_date, '賣出價': sell_price, '股數': info['shares'],
                    '損益': (sell_price - info['buy_price']) * info['shares'],
                    '報酬率': (sell_price / info['buy_price']) - 1,
                    'SMA參數': sma_period, '原因': '再平衡/換股'
                })
                cash += info['shares'] * sell_price

            total_val = cash
            if target_tickers:
                alloc_per_stock = total_val / len(target_tickers)
                for t in target_tickers:
                    p = prices.loc[trade_date, t]
                    s = alloc_per_stock // p
                    if s > 0:
                        holdings[t] = {
                            'shares': s, 'buy_date': trade_date,
                            'buy_signal_date': signal_date_t, 'buy_price': p
                        }
                        cash -= s * p
            
            pending_rebalance = False
            daily_records[-1]['事件'] = '執行交易'
            daily_records[-1]['細節'] = f'持有: {list(holdings.keys())}'

        daily_holdings_log.append({
            '日期': current_date,
            '持股檔數': len(holdings),
            '持股明細': ", ".join(holdings.keys())
        })

    return equity, closed_trades, daily_holdings_log, daily_records

def calculate_metrics(equity):
    total_return = (equity.iloc[-1] / equity.iloc[0]) - 1
    days = (equity.index[-1] - equity.index[0]).days
    if days <= 0: return 0, 0, 0
    cagr = (1 + total_return) ** (365.25 / days) - 1
    rolling_max = equity.cummax()
    drawdown = (equity - rolling_max) / rolling_max
    max_dd = drawdown.min()
    calmar = cagr / abs(max_dd) if max_dd != 0 else 0
    return cagr, max_dd, calmar

def optimize_sma(prices, ticker_to_name):
    best_calmar = -1
    best_sma = 0
    plateau = []
    print("開始最佳化 SMA 參數...")
    for s in range(10, 251, 10):
        equity, _, _, _ = run_backtest_full(prices, s, ticker_to_name)
        cagr, max_dd, calmar = calculate_metrics(equity)
        plateau.append({'SMA': s, 'CAGR': cagr, 'MaxDD': max_dd, 'Calmar': calmar})
        print(f"SMA: {s}, CAGR: {cagr:.2%}, MaxDD: {max_dd:.2%}, Calmar: {calmar:.2f}")
        if calmar > best_calmar:
            best_calmar = calmar
            best_sma = s
    return best_sma, plateau

def main():
    if not os.path.exists('data/data.xlsx'):
        print("找不到資料檔案 data/data.xlsx")
        return
    prices, ticker_to_name = load_and_preprocess_data(FILE_PATH)
    best_sma, plateau_data = optimize_sma(prices, ticker_to_name)
    print(f"\n最佳參數: SMA {best_sma}")
    equity, trades, hold_log, daily_rec = run_backtest_full(prices, best_sma, ticker_to_name)
    cagr, max_dd, calmar = calculate_metrics(equity)
    trades_df = pd.DataFrame(trades)
    win_rate = (trades_df['損益'] > 0).mean() if not trades_df.empty else 0
    
    with pd.ExcelWriter(RESULT_EXCEL, engine='xlsxwriter') as writer:
        trades_df.to_excel(writer, sheet_name='Trades', index=False)
        ec_df = pd.DataFrame({'Equity': equity})
        ec_df['Drawdown'] = (equity - equity.cummax()) / equity.cummax()
        ec_df.to_excel(writer, sheet_name='Equity_Curve')
        pd.DataFrame(hold_log).to_excel(writer, sheet_name='Equity_Hold', index=False)
        pd.DataFrame(daily_rec).to_excel(writer, sheet_name='Daily_Record', index=False)
        summary = pd.DataFrame({
            '指標': ['最佳 SMA 參數', '年化報酬率 (CAGR)', '最大回撤 (MaxDD)', 'Calmar Ratio', '勝率', '最終資產'],
            '數值': [best_sma, f"{cagr:.2%}", f"{max_dd:.2%}", f"{calmar:.2f}", f"{win_rate:.2%}", f"{equity.iloc[-1]:,.0f}"]
        })
        summary.to_excel(writer, sheet_name='Summary', index=False)
        pd.DataFrame(plateau_data).to_excel(writer, sheet_name='Plateau_Table', index=False)
        
        workbook = writer.book
        worksheet = writer.sheets['Equity_Curve']
        chart = workbook.add_chart({'type': 'line'})
        n_rows = len(ec_df)
        chart.add_series({
            'name': 'Equity',
            'categories': ['Equity_Curve', 1, 0, n_rows, 0],
            'values': ['Equity_Curve', 1, 1, n_rows, 1],
        })
        chart.set_title({'name': 'Equity Curve'})
        worksheet.insert_chart('E2', chart)

    formula_sma = r"SMA_n = \\frac{1}{n} \sum_{i=0}^{n-1} P_{t-i}"
    formula_roc = r"ROC_n = \\frac{P_t - P_{t-n}}{P_{t-n}}"
    formula_cagr = r"(\\frac{Final Equity}{Initial Equity})^{365.25 / Days} - 1"
    
    with open(REPRODUCE_MD, 'w', encoding='utf-8') as f:
        f.write(f"""# Asset Class Trend Following 策略重現指南

## 1. 策略邏輯說明
本策略參考 QuantConnect Strategy 17 "Asset Class Trend Following"。
- **資產池**: 123 檔標的。
- **指標**: 簡單移動平均線 (SMA) 與 變動率 (ROC)。
- **進場條件**: 當價格高於 SMA 時，視為上升趨勢。
- **排名依據**: 在所有符合進場條件的標的中，依據 ROC (動能) 由高到低排名。
- **持股數量**: 最多持有 {MAX_HOLDINGS} 檔。
- **權重分配**: 等權重分配 (1/N)。
- **再平衡週期**: 每 5 個交易日。
- **交易執行**: T 日產生訊號，T+1 日以收盤價執行交易。

## 2. 計算公式
- **SMA (Simple Moving Average)**: ${formula_sma}$
- **ROC (Rate of Change)**: ${formula_roc}$
- **CAGR (Compound Annual Growth Rate)**: ${formula_cagr}$
- **Max Drawdown**: $min(\\frac{{Equity - Peak}}{{Peak}})$
- **Calmar Ratio**: $CAGR / |MaxDD|$

## 3. 最佳化結果
- **最佳 SMA 參數**: {best_sma}
- **CAGR**: {cagr:.2%}
- **MaxDD**: {max_dd:.2%}
- **Calmar Ratio**: {calmar:.2f}

## 4. 參數高原 (部分)
| SMA | CAGR | MaxDD | Calmar |
|-----|------|-------|--------|
""")
        for p in plateau_data[:10]:
            f.write(f"| {p['SMA']} | {p['CAGR']:.2%} | {p['MaxDD']:.2%} | {p['Calmar']:.2f} |\n")
        f.write("\n(完整高原表請詳見 Excel 檔案中的 Plateau_Table 分頁)\n")
    print(f"\n所有結果已儲存至 {RESULT_EXCEL} 與 {REPRODUCE_MD}")

if __name__ == "__main__":
    main()


開始最佳化 SMA 參數...


SMA: 10, CAGR: 86.32%, MaxDD: -52.66%, Calmar: 1.64


SMA: 20, CAGR: 86.20%, MaxDD: -57.91%, Calmar: 1.49


SMA: 30, CAGR: 111.47%, MaxDD: -39.00%, Calmar: 2.86


SMA: 40, CAGR: 104.69%, MaxDD: -38.92%, Calmar: 2.69


...

SMA: 220, CAGR: 77.06%, MaxDD: -59.47%, Calmar: 1.30


SMA: 230, CAGR: 90.97%, MaxDD: -54.77%, Calmar: 1.66


SMA: 240, CAGR: 85.06%, MaxDD: -54.77%, Calmar: 1.55


SMA: 250, CAGR: 74.07%, MaxDD: -54.77%, Calmar: 1.35

最佳參數: SMA 50



所有結果已儲存至 strategy_results.xlsx 與 reproduce_strategy.md
